In [1]:
import pandas as pd
import numpy as np
import json
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import os
from konlpy.tag import Hannanum
import requests
import re
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import matplotlib.font_manager as fm

In [9]:
okt = Okt()

In [3]:
news = pd.read_excel("project1/data/mk_news_201901_202006.xlsx")

In [4]:
news['pubdate'] = pd.to_datetime(news['pubdate'], format = '%Y%m%d', errors = 'raise')

In [4]:
news.head()

,no,pubdate,pubtime,title1,body,keywords
0,2304119,2020-03-12,0.0,사회일반,NaN,NaN
1,3937779,2019-01-12,0.0,출판/도서,<!--[[--image0--]]//--> ◆ 야바위 게임 / 마이클 슈월비 지음...,"신간,소개"
2,3953320,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 / 부동산 ◆ \n <!--[[--image0--]]//--...,"ㅈ,ㅈ"
3,3953321,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 / 금리 ◆ \n <!--[[--image0--]]//-->...,"금리,경제기상도"
4,3953324,2019-01-01,0.0,테마기획,<!--[[--image0--]]//--> '황금돼지해'인 2019년 기해년(己亥...,"경제,성장,경제기상도"


# 2019.02 데이터

In [5]:
news_201902 = news[(news['pubdate']>='2019-02-01') & (news['pubdate']<'2019-03-01')]
news_201902

,no,pubdate,pubtime,title1,body,keywords
16707,3984663,2019-02-01,0.0,테마기획,<!--[[--image0--]]//--> 올해 설 연휴 귀성길은 예년에 비해 교...,"설,귀성"
16708,3984665,2019-02-01,0.0,테마기획,<!--[[--image0--]]//--> 설 연휴 기간에 귀성 혹은 역귀성을 위...,"보,험"
16709,3984666,2019-02-01,0.0,테마기획,"<!--[[--image0--]]//--> '어른들에게는 어릴 적 향수를, 아이들...","설특집,가볼만한곳"
16710,3984667,2019-02-01,0.0,테마기획,"<!--[[--image0--]]//--> ""생선 머리는 어느 쪽을 바라보게 둬야...","설,서비스"
16711,3984668,2019-02-01,0.0,테마기획,설 연휴 기간 응급의료가 필요한 상황이 닥칠 경우 119를 이용하면 된다. 소방청은...,"설 연휴,응급환자"
...,...,...,...,...,...,...
30898,4012550,2019-02-28,40023.0,의약/건강,<!--[[--image0--]]//--> 국제 공동 연구진이 미숙아의 혈압이나...,"미숙아,센서"
31616,4014340,2019-02-28,93304.0,증권계동향,증시가 다양한 호재와 악재의 혼재로 확실한 방향성을 정하지 못하고 있는 모습이다. ...,NaN
31624,4014351,2019-02-28,93355.0,증권계동향,"전문가들은 ""코스피의 움직임이 제한적인 가운데 종목별, 업종별 이슈 중심의 순환매가...",NaN
31625,4014352,2019-02-28,93436.0,증권계동향,전문가들은 코스피가 3월에도 오름세는 이어질 것으로 예상되지만 여전히 관망세에 무게...,NaN


In [6]:
news_201902['body1'] = news_201902['body'].apply(lambda x : re.sub('[0-9]+', '', x))
news_201902['body1'] = news_201902['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
news_201902['body1'] = news_201902['body1'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》◆■\n]', '', x))

<ipython-input-6-70ed802fb590>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201902['body1'] = news_201902['body'].apply(lambda x : re.sub('[0-9]+', '', x))
<ipython-input-6-70ed802fb590>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201902['body1'] = news_201902['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
<ipython-input-6-70ed802fb590>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [7]:
news_201902

,no,pubdate,pubtime,title1,body,keywords,body1
16707,3984663,2019-02-01,0.0,테마기획,<!--[[--image0--]]//--> 올해 설 연휴 귀성길은 예년에 비해 교...,"설,귀성",올해 설 연휴 귀성길은 예년에 비해 교통체증이 한결 덜할 것으로 전망된다 지난해...
16708,3984665,2019-02-01,0.0,테마기획,<!--[[--image0--]]//--> 설 연휴 기간에 귀성 혹은 역귀성을 위...,"보,험",설 연휴 기간에 귀성 혹은 역귀성을 위해 자동차 운전대를 잡는 일이 많다 이 경...
16709,3984666,2019-02-01,0.0,테마기획,"<!--[[--image0--]]//--> '어른들에게는 어릴 적 향수를, 아이들...","설특집,가볼만한곳",어른들에게는 어릴 적 향수를 아이들에게는 잊지 못할 추억을이번 설연휴 기간에도 ...
16710,3984667,2019-02-01,0.0,테마기획,"<!--[[--image0--]]//--> ""생선 머리는 어느 쪽을 바라보게 둬야...","설,서비스",생선 머리는 어느 쪽을 바라보게 둬야 할까민족 대명절 설이 돌아왔다 이번 설 연...
16711,3984668,2019-02-01,0.0,테마기획,설 연휴 기간 응급의료가 필요한 상황이 닥칠 경우 119를 이용하면 된다. 소방청은...,"설 연휴,응급환자",설 연휴 기간 응급의료가 필요한 상황이 닥칠 경우 를 이용하면 된다 소방청은 설 연...
...,...,...,...,...,...,...,...
30898,4012550,2019-02-28,40023.0,의약/건강,<!--[[--image0--]]//--> 국제 공동 연구진이 미숙아의 혈압이나...,"미숙아,센서",국제 공동 연구진이 미숙아의 혈압이나 심박수 등을 무선으로 측정할 수 있는 기...
31616,4014340,2019-02-28,93304.0,증권계동향,증시가 다양한 호재와 악재의 혼재로 확실한 방향성을 정하지 못하고 있는 모습이다. ...,NaN,증시가 다양한 호재와 악재의 혼재로 확실한 방향성을 정하지 못하고 있는 모습이다 특...
31624,4014351,2019-02-28,93355.0,증권계동향,"전문가들은 ""코스피의 움직임이 제한적인 가운데 종목별, 업종별 이슈 중심의 순환매가...",NaN,전문가들은 코스피의 움직임이 제한적인 가운데 종목별 업종별 이슈 중심의 순환매가 예...
31625,4014352,2019-02-28,93436.0,증권계동향,전문가들은 코스피가 3월에도 오름세는 이어질 것으로 예상되지만 여전히 관망세에 무게...,NaN,전문가들은 코스피가 월에도 오름세는 이어질 것으로 예상되지만 여전히 관망세에 무게를...


In [9]:
news_201902['freq'] = news_201902['body1'].apply(lambda x : okt.nouns(x))

<ipython-input-9-e90574646aa2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201902['freq'] = news_201902['body1'].apply(lambda x : okt.nouns(x))


In [10]:
news_201902['freq_10'] = news_201902['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))

<ipython-input-10-a03eaeff590f>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  news_201902['freq_10'] = news_201902['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))
<ipython-input-10-a03eaeff590f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201902['freq_10'] = news_201902['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))


In [14]:
news_201902['freq_10'] = news_201902['freq_10'].apply(lambda x : ",".join(x))

<ipython-input-14-79382d2b50b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201902['freq_10'] = news_201902['freq_10'].apply(lambda x : ",".join(x))


In [29]:
news_201902

,no,pubdate,pubtime,title1,body,keywords,body1,freq,freq_10
16707,3984663,2019-02-01,0.0,테마기획,<!--[[--image0--]]//--> 올해 설 연휴 귀성길은 예년에 비해 교...,"설,귀성",올해 설 연휴 귀성길은 예년에 비해 교통체증이 한결 덜할 것으로 전망된다 지난해...,"[올해, 설, 연휴, 귀성, 예년, 교통체증, 한결, 것, 전망, 지난해, 기간, ...","시간,일,월,등,것,서울,교통,예상,귀성,운행"
16708,3984665,2019-02-01,0.0,테마기획,<!--[[--image0--]]//--> 설 연휴 기간에 귀성 혹은 역귀성을 위...,"보,험",설 연휴 기간에 귀성 혹은 역귀성을 위해 자동차 운전대를 잡는 일이 많다 이 경...,"[설, 연휴, 기간, 귀성, 역, 귀성, 위해, 자동차, 운전대, 이, 경우, 운전...","서비스,점검,가입,경우,일,손보,차량,무상,운전,보험"
16709,3984666,2019-02-01,0.0,테마기획,"<!--[[--image0--]]//--> '어른들에게는 어릴 적 향수를, 아이들...","설특집,가볼만한곳",어른들에게는 어릴 적 향수를 아이들에게는 잊지 못할 추억을이번 설연휴 기간에도 ...,"[어른, 적, 향수, 아이, 추억, 이번, 설연휴, 기간, 전국, 윷놀이, 널뛰기,...","등,일,체험,행사,윷놀이,수,시,전통,연휴,설"
16710,3984667,2019-02-01,0.0,테마기획,"<!--[[--image0--]]//--> ""생선 머리는 어느 쪽을 바라보게 둬야...","설,서비스",생선 머리는 어느 쪽을 바라보게 둬야 할까민족 대명절 설이 돌아왔다 이번 설 연...,"[생선, 머리, 쪽, 민족, 명절, 설, 이번, 설, 연휴, 주말, 포함, 무려, ...","앱,수,연휴,설,명절,정보,것,차례상,시간,법"
16711,3984668,2019-02-01,0.0,테마기획,설 연휴 기간 응급의료가 필요한 상황이 닥칠 경우 119를 이용하면 된다. 소방청은...,"설 연휴,응급환자",설 연휴 기간 응급의료가 필요한 상황이 닥칠 경우 를 이용하면 된다 소방청은 설 연...,"[설, 연휴, 기간, 응급의료, 상황, 경우, 를, 이용, 소방청, 설, 연휴, 기...","약국,연휴,기간,병원,기관,등,설,의료,응급의료,상황"
...,...,...,...,...,...,...,...,...,...
30898,4012550,2019-02-28,40023.0,의약/건강,<!--[[--image0--]]//--> 국제 공동 연구진이 미숙아의 혈압이나...,"미숙아,센서",국제 공동 연구진이 미숙아의 혈압이나 심박수 등을 무선으로 측정할 수 있는 기...,"[국제, 공동, 구진, 미숙아, 혈압, 심박수, 등, 무선, 측정, 수, 기술, 개...","센서,아기,무선,수,구진,등,신생아,기존,개발,혈압"
31616,4014340,2019-02-28,93304.0,증권계동향,증시가 다양한 호재와 악재의 혼재로 확실한 방향성을 정하지 못하고 있는 모습이다. ...,NaN,증시가 다양한 호재와 악재의 혼재로 확실한 방향성을 정하지 못하고 있는 모습이다 특...,"[증시, 호재, 악, 재의, 혼재, 방향성, 정, 못, 모습, 밸류, 션, 부담, ...","탁론,종목,호스,수,금리,가능,상품,수수료,이용,투자자"
31624,4014351,2019-02-28,93355.0,증권계동향,"전문가들은 ""코스피의 움직임이 제한적인 가운데 종목별, 업종별 이슈 중심의 순환매가...",NaN,전문가들은 코스피의 움직임이 제한적인 가운데 종목별 업종별 이슈 중심의 순환매가 예...,"[전문가, 코스피, 움직임, 제한, 가운데, 종목, 별, 업종, 별, 이슈, 중심,...","종목,탁론,스,골드,투자자,수,상담,금리,대출,투자"
31625,4014352,2019-02-28,93436.0,증권계동향,전문가들은 코스피가 3월에도 오름세는 이어질 것으로 예상되지만 여전히 관망세에 무게...,NaN,전문가들은 코스피가 월에도 오름세는 이어질 것으로 예상되지만 여전히 관망세에 무게를...,"[전문가, 코스피, 월, 것, 예상, 관망, 세, 무게, 다만, 대외, 이벤트, 대...","탁론,수,종목,호스,금리,것,투자자,대출,이용,가능"


In [32]:
news_201902.to_excel("news_201902.xlsx")

# 2019.01 데이터

In [20]:
news_201901 = news[(news['pubdate']>='2019-01-01') & (news['pubdate']<'2019-02-01')]
news_201901

,no,pubdate,pubtime,title1,body,keywords
1,3937779,2019-01-12,0.0,출판/도서,<!--[[--image0--]]//--> ◆ 야바위 게임 / 마이클 슈월비 지음...,"신간,소개"
2,3953320,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 / 부동산 ◆ \n <!--[[--image0--]]//--...,"ㅈ,ㅈ"
3,3953321,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 / 금리 ◆ \n <!--[[--image0--]]//-->...,"금리,경제기상도"
4,3953324,2019-01-01,0.0,테마기획,<!--[[--image0--]]//--> '황금돼지해'인 2019년 기해년(己亥...,"경제,성장,경제기상도"
5,3953326,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 ◆ \n <!--[[--image0--]]//--> 섬유업계...,"섬유,업계"
...,...,...,...,...,...,...
17366,3985796,2019-01-31,232040.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 이누리 객원기자]...,NaN
17367,3985797,2019-01-31,232226.0,방송/TV,<!--[[--image0--]]//--> [ 매일경제 스타투데이 허은경 객원기자...,NaN
17369,3985805,2019-01-31,233923.0,핫이슈,<!--[[--image0--]]//--> [매일경제 스타투데이 한현정 기자]\n...,NaN
17370,3985806,2019-01-31,234727.0,방송/TV,<!--[[--image0--]]//--> [ 매일경제 스타투데이 허은경 객원기자...,NaN


In [21]:
news_201901['body1'] = news_201901['body'].apply(lambda x : re.sub('[0-9]+', '', x))
news_201901['body1'] = news_201901['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
news_201901['body1'] = news_201901['body1'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》◆■\n]', '', x))

<ipython-input-21-0a277393e451>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201901['body1'] = news_201901['body'].apply(lambda x : re.sub('[0-9]+', '', x))
<ipython-input-21-0a277393e451>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201901['body1'] = news_201901['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
<ipython-input-21-0a277393e451>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [22]:
news_201901

,no,pubdate,pubtime,title1,body,keywords,body1
1,3937779,2019-01-12,0.0,출판/도서,<!--[[--image0--]]//--> ◆ 야바위 게임 / 마이클 슈월비 지음...,"신간,소개",야바위 게임 마이클 슈월비 지음 노정태 옮김 만원 도덕의 문제로 불평등의...
2,3953320,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 / 부동산 ◆ \n <!--[[--image0--]]//--...,"ㅈ,ㅈ",경제기상도 부동산 년 부동산 시장은 롤러코스터를 타듯 요동친 한 해였다 ...
3,3953321,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 / 금리 ◆ \n <!--[[--image0--]]//-->...,"금리,경제기상도",경제기상도 금리 명분도 무엇보다 여력도 없다 원칙적으로 금리 인상은 경기...
4,3953324,2019-01-01,0.0,테마기획,<!--[[--image0--]]//--> '황금돼지해'인 2019년 기해년(己亥...,"경제,성장,경제기상도",황금돼지해인 년 기해년己亥年 대한민국 경제가 다시 활력을 되찾을지 관심이 쏠린다...
5,3953326,2019-01-01,0.0,테마기획,◆ 2019 경제기상도 ◆ \n <!--[[--image0--]]//--> 섬유업계...,"섬유,업계",경제기상도 섬유업계는 지난해 미중 무역분쟁과 유가 하락 경쟁 심화로 인한 ...
...,...,...,...,...,...,...,...
17366,3985796,2019-01-31,232040.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 이누리 객원기자]...,NaN,매일경제 스타투데이 이누리 객원기자 인생술집 모델 한혜진이 동명이인이 많아서 겪...
17367,3985797,2019-01-31,232226.0,방송/TV,<!--[[--image0--]]//--> [ 매일경제 스타투데이 허은경 객원기자...,NaN,매일경제 스타투데이 허은경 객원기자 해피투게더 배우 조병규가 만만찮은 토크 내...
17369,3985805,2019-01-31,233923.0,핫이슈,<!--[[--image0--]]//--> [매일경제 스타투데이 한현정 기자]\n...,NaN,매일경제 스타투데이 한현정 기자배우 최민수가 보복운전 혐의로 기소돼 재판에 넘겨...
17370,3985806,2019-01-31,234727.0,방송/TV,<!--[[--image0--]]//--> [ 매일경제 스타투데이 허은경 객원기자...,NaN,매일경제 스타투데이 허은경 객원기자 해피투게더 김보라와 조병규가 열애설에 대해...


In [23]:
news_201901['freq'] = news_201901['body1'].apply(lambda x : okt.nouns(x))

<ipython-input-23-0e3d2cecc9cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201901['freq'] = news_201901['body1'].apply(lambda x : okt.nouns(x))


In [24]:
news_201901['freq_10'] = news_201901['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))

<ipython-input-24-31a3e3b5e863>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  news_201901['freq_10'] = news_201901['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))
<ipython-input-24-31a3e3b5e863>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201901['freq_10'] = news_201901['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))


In [25]:
news_201901['freq_10'] = news_201901['freq_10'].apply(lambda x : ",".join(x))

<ipython-input-25-f1ada32c9e0c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201901['freq_10'] = news_201901['freq_10'].apply(lambda x : ",".join(x))


In [34]:
news_201901.to_excel("news_201901.xlsx")

# 2019.03 데이터

In [26]:
news_201903 = news[(news['pubdate']>='2019-03-01') & (news['pubdate']<'2019-04-01')]
news_201903

,no,pubdate,pubtime,title1,body,keywords
29618,4010109,2019-03-04,0.0,중견/중소기업,평생교육 대표 기업 휴넷(대표 조영탁)이 공인중개사 시험에 합격할 때까지 평생 수강...,"휴넷,공인중개사"
30545,4011800,2019-03-01,0.0,법원·검찰·경찰,관세청 인사에 개입하고 '뒷돈'을 챙긴 혐의로 재판에 넘겨진 최순실 씨의 옛 지인 ...,"고영태,대법"
30554,4011815,2019-03-01,0.0,법원·검찰·경찰,생후 8개월 아들을 때려 숨지게 한 뒤 시신을 방치한 40대 여성에게 대법원이 징역...,"법원,대법"
30557,4011823,2019-03-01,0.0,인물/인터뷰,"""평생 등이 굽은 채로 살아가야 할 줄 알았는데 이렇게 치료해주셔서 너무나 감사합니...","아프리카,아프리카 마다가스카르,마다가스카르,한림대동탄성심병원,동탄성심병원,나린드라,..."
30559,4011828,2019-03-01,0.0,증권계동향,3월 미래에셋생명·하나금융지주 등 일부 지분에 대한 의무보호예수가 해제된다. 이에 ...,"예탁결제원,미래에셋생명,의무보호예수"
...,...,...,...,...,...,...
47850,4046097,2019-03-29,105535.0,증권계동향,반도체 수요가 2분기 바닥을 찍고 3분기부터 다시 회복세로 돌아설 것이라는 전망이 ...,NaN
47851,4046098,2019-03-29,105721.0,증권계동향,오는 10일-11일 한·미 정상회담 개최 소식에 남북경협주가 상승세를 보이고 있다....,NaN
47904,4046180,2019-03-29,105802.0,증권계동향,국내 주요 증권사 리서치센터장들은 올해 2분기 코스피 시장이 기업 실적 부진과 경기...,NaN
47905,4046181,2019-03-29,105850.0,증권계동향,증시 전문가들은 2분기 코스피 시장이 2100~2300선에서 움직이는 박스권 장세를...,NaN


In [27]:
news_201903['body1'] = news_201903['body'].apply(lambda x : re.sub('[0-9]+', '', x))
news_201903['body1'] = news_201903['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
news_201903['body1'] = news_201903['body1'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》◆■\n]', '', x))

<ipython-input-27-0c657a05e457>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201903['body1'] = news_201903['body'].apply(lambda x : re.sub('[0-9]+', '', x))
<ipython-input-27-0c657a05e457>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201903['body1'] = news_201903['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
<ipython-input-27-0c657a05e457>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [28]:
news_201903['freq'] = news_201903['body1'].apply(lambda x : okt.nouns(x))

<ipython-input-28-84dfad82c4b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201903['freq'] = news_201903['body1'].apply(lambda x : okt.nouns(x))


In [35]:
news_201903['freq_10'] = news_201903['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))

<ipython-input-35-22e634b70efb>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  news_201903['freq_10'] = news_201903['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))
<ipython-input-35-22e634b70efb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201903['freq_10'] = news_201903['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))


In [36]:
news_201903['freq_10'] = news_201903['freq_10'].apply(lambda x : ",".join(x))

<ipython-input-36-1ae4508970dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_201903['freq_10'] = news_201903['freq_10'].apply(lambda x : ",".join(x))


In [38]:
news_201903.to_excel("news_201903.xlsx")

# 2020.02 데이터

In [39]:
news_202002 = news[(news['pubdate']>='2020-02-01') & (news['pubdate']<'2020-03-01')]
news_202002

,no,pubdate,pubtime,title1,body,keywords
222119,4376729,2020-02-01,0.0,교육,정부가 대학 기술지주회사의 자회사 주식 보유 한도를 20%에서 10%로 낮추고 보유...,"교육,서울대,기술지주회사"
222360,4377141,2020-02-01,0.0,사회일반,지난해 서울 한 사립대에 입학한 박 모씨(20)는 최근 신입생 후배들을 맞으면서 과...,"젊은꼰대,대학,신입생"
222425,4377242,2020-02-01,0.0,은행,<!--[[--image0--]]//--> 제24대 농협중앙회 회장에 이성희 전 ...,"ㅇ,ㅇ"
222504,4377378,2020-02-01,0.0,법원·검찰·경찰,서울시교육청이 한국유치원총연합회(한유총)에 내린 법인 설립허가 취소처분이 부당하다는...,"법원,한유총"
222507,4377382,2020-02-01,0.0,지역경제,서울시가 위례신사선 도시철도 민간투자사업 우선협상대상자로 가칭 강남메트로주식회사(주...,"위례신사선,우선협상대상자,강남메트로"
...,...,...,...,...,...,...
239813,4407568,2020-02-29,223101.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 원태성 객원기자]...,NaN
239814,4407569,2020-02-29,223433.0,NaN,베트남 당국이 하노이 공항에 이어 호찌민 공항에도 한국발 여객기 착륙을 불허하기로 ...,NaN
239815,4407571,2020-02-29,223529.0,국제사회,"[속보] 미국-아프간 탈레반, 평화합의 타결\t\n[디지털뉴스국]",NaN
239816,4407572,2020-02-29,223727.0,사회일반,중국에서 신종 코로나바이러스 감염증(코로나19)으로 사망한 환자를 부검한 결과 중증...,NaN


In [40]:
news_202002['body1'] = news_202002['body'].apply(lambda x : re.sub('[0-9]+', '', x))
news_202002['body1'] = news_202002['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
news_202002['body1'] = news_202002['body1'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》◆■\n]', '', x))

<ipython-input-40-fed76ae80a11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202002['body1'] = news_202002['body'].apply(lambda x : re.sub('[0-9]+', '', x))
<ipython-input-40-fed76ae80a11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202002['body1'] = news_202002['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
<ipython-input-40-fed76ae80a11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [41]:
news_202002

,no,pubdate,pubtime,title1,body,keywords,body1
222119,4376729,2020-02-01,0.0,교육,정부가 대학 기술지주회사의 자회사 주식 보유 한도를 20%에서 10%로 낮추고 보유...,"교육,서울대,기술지주회사",정부가 대학 기술지주회사의 자회사 주식 보유 한도를 에서 로 낮추고 보유의무 유예기...
222360,4377141,2020-02-01,0.0,사회일반,지난해 서울 한 사립대에 입학한 박 모씨(20)는 최근 신입생 후배들을 맞으면서 과...,"젊은꼰대,대학,신입생",지난해 서울 한 사립대에 입학한 박 모씨는 최근 신입생 후배들을 맞으면서 과거 유쾌...
222425,4377242,2020-02-01,0.0,은행,<!--[[--image0--]]//--> 제24대 농협중앙회 회장에 이성희 전 ...,"ㅇ,ㅇ",제대 농협중앙회 회장에 이성희 전 낙생농협 조합장사진이 당선됐다 일 서울 중구 ...
222504,4377378,2020-02-01,0.0,법원·검찰·경찰,서울시교육청이 한국유치원총연합회(한유총)에 내린 법인 설립허가 취소처분이 부당하다는...,"법원,한유총",서울시교육청이 한국유치원총연합회한유총에 내린 법인 설립허가 취소처분이 부당하다는 법...
222507,4377382,2020-02-01,0.0,지역경제,서울시가 위례신사선 도시철도 민간투자사업 우선협상대상자로 가칭 강남메트로주식회사(주...,"위례신사선,우선협상대상자,강남메트로",서울시가 위례신사선 도시철도 민간투자사업 우선협상대상자로 가칭 강남메트로주식회사주간...
...,...,...,...,...,...,...,...
239813,4407568,2020-02-29,223101.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 원태성 객원기자]...,NaN,매일경제 스타투데이 원태성 객원기자차유리김태희 분이 오민정고보결 분과 마주쳤다일...
239814,4407569,2020-02-29,223433.0,NaN,베트남 당국이 하노이 공항에 이어 호찌민 공항에도 한국발 여객기 착륙을 불허하기로 ...,NaN,베트남 당국이 하노이 공항에 이어 호찌민 공항에도 한국발 여객기 착륙을 불허하기로 ...
239815,4407571,2020-02-29,223529.0,국제사회,"[속보] 미국-아프간 탈레반, 평화합의 타결\t\n[디지털뉴스국]",NaN,속보 미국아프간 탈레반 평화합의 타결\t디지털뉴스국
239816,4407572,2020-02-29,223727.0,사회일반,중국에서 신종 코로나바이러스 감염증(코로나19)으로 사망한 환자를 부검한 결과 중증...,NaN,중국에서 신종 코로나바이러스 감염증코로나으로 사망한 환자를 부검한 결과 중증급성호흡...


In [42]:
news_202002['freq'] = news_202002['body1'].apply(lambda x : okt.nouns(x))

<ipython-input-42-fa4ce5d491f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202002['freq'] = news_202002['body1'].apply(lambda x : okt.nouns(x))


In [43]:
news_202002['freq_10'] = news_202002['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))

<ipython-input-43-b42e55b5a98f>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  news_202002['freq_10'] = news_202002['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))
<ipython-input-43-b42e55b5a98f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202002['freq_10'] = news_202002['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))


In [44]:
news_202002['freq_10'] = news_202002['freq_10'].apply(lambda x : ",".join(x))

<ipython-input-44-49fdf987bd1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202002['freq_10'] = news_202002['freq_10'].apply(lambda x : ",".join(x))


In [46]:
news_202002.to_excel("news_202002.xlsx")

# 2020.01 데이터

In [5]:
news_202001 = news[(news['pubdate']>='2020-01-01') & (news['pubdate']<'2020-02-01')]
news_202001

,no,pubdate,pubtime,title1,body,keywords
205299,4345606,2020-01-01,0.0,테마기획,<!--[[--image0--]]//--> 2019년은 '황금돼지해'란 말이 무색...,"경제전망,2020년"
205302,4345609,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 증시 ◆\n <!--[[--image0--]]//--> ...,"증시,주식,시장,전망,2020,투자,금융,증권,증시,환율,"
205303,4345610,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 환율 ◆\n <!--[[--image0--]]//--> ...,"환율,달러당원화값,미중 무역갈등"
205304,4345611,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 부동산 ◆\n <!--[[--image0--]]//-->...,"ㅐ,ㅐ"
205305,4345612,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 금리 ◆\n <!--[[--image0--]]//--> ...,"금리,2020"
...,...,...,...,...,...,...
222726,4377749,2020-01-31,220944.0,국제정치,세계보건기구(WHO)는 각국을 향해 신종 코로나바이러스 확산에 대응해 국경을 폐쇄해...,NaN
222727,4377753,2020-01-31,222643.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN
222728,4377760,2020-01-31,230012.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN
222729,4377765,2020-01-31,233038.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN


In [6]:
news_202001['body1'] = news_202001['body'].apply(lambda x : re.sub('[0-9]+', '', x))
news_202001['body1'] = news_202001['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
news_202001['body1'] = news_202001['body1'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》◆■\n]', '', x))

<ipython-input-6-44b3afa29470>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202001['body1'] = news_202001['body'].apply(lambda x : re.sub('[0-9]+', '', x))
<ipython-input-6-44b3afa29470>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202001['body1'] = news_202001['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
<ipython-input-6-44b3afa29470>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [7]:
news_202001

,no,pubdate,pubtime,title1,body,keywords,body1
205299,4345606,2020-01-01,0.0,테마기획,<!--[[--image0--]]//--> 2019년은 '황금돼지해'란 말이 무색...,"경제전망,2020년",년은 황금돼지해란 말이 무색한 한 해였다 지난해 정부는 우리나라 실질 국내총생산...
205302,4345609,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 증시 ◆\n <!--[[--image0--]]//--> ...,"증시,주식,시장,전망,2020,투자,금융,증권,증시,환율,",경제기상도 증시 한국 증시를 짓누르던 미중 무역분쟁이 단계 합의로 소강 상...
205303,4345610,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 환율 ◆\n <!--[[--image0--]]//--> ...,"환율,달러당원화값,미중 무역갈등",경제기상도 환율 지난해 미중 무역분쟁이라는 대형 변수로 출렁였던 외환시장은...
205304,4345611,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 부동산 ◆\n <!--[[--image0--]]//-->...,"ㅐ,ㅐ",경제기상도 부동산 년 말 대다수 부동산 전문가들은 년 서울 집값이 하락할 ...
205305,4345612,2020-01-01,0.0,테마기획,◆ 2020 경제기상도 / 금리 ◆\n <!--[[--image0--]]//--> ...,"금리,2020",경제기상도 금리 올해 기준금리 운용의 핵심 키워드는 완화 기조 유지다 한국...
...,...,...,...,...,...,...,...
222726,4377749,2020-01-31,220944.0,국제정치,세계보건기구(WHO)는 각국을 향해 신종 코로나바이러스 확산에 대응해 국경을 폐쇄해...,NaN,세계보건기구는 각국을 향해 신종 코로나바이러스 확산에 대응해 국경을 폐쇄해서는 안 ...
222727,4377753,2020-01-31,222643.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN,매일경제 스타투데이 서지경 객원기자 금금밤에서 대 플로리스트이자 주부인 노은수...
222728,4377760,2020-01-31,230012.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN,매일경제 스타투데이 서지경 객원기자 금금밤에서 오상진 어머니가 소녀 같은 모습...
222729,4377765,2020-01-31,233038.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN,매일경제 스타투데이 서지경 객원기자 나혼자산다에서 조병규가 우연히 윤병희 배우...


In [10]:
news_202001['freq'] = news_202001['body1'].apply(lambda x : okt.nouns(x))

<ipython-input-10-16703dc9b1cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202001['freq'] = news_202001['body1'].apply(lambda x : okt.nouns(x))


In [11]:
news_202001['freq_10'] = news_202001['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))

<ipython-input-11-84bb2f0422b9>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  news_202001['freq_10'] = news_202001['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))
<ipython-input-11-84bb2f0422b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202001['freq_10'] = news_202001['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))


In [12]:
news_202001['freq_10'] = news_202001['freq_10'].apply(lambda x : ",".join(x))

<ipython-input-12-3d0085474cc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202001['freq_10'] = news_202001['freq_10'].apply(lambda x : ",".join(x))


In [13]:
news_202001.to_excel("news_202001.xlsx")

# 2020.03 데이터

In [14]:
news_202003 = news[(news['pubdate']>='2020-03-01') & (news['pubdate']<'2020-04-01')]
news_202003

,no,pubdate,pubtime,title1,body,keywords
0,2304119,2020-03-12,0.0,사회일반,NaN,NaN
239818,4407584,2020-03-01,2159.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 원태성 객원기자]...,NaN
239819,4407589,2020-03-01,41109.0,영화,<!--[[--image0--]]//--> \n[매일경제 스타투데이 양소영 기자...,NaN
239820,4407590,2020-03-01,43011.0,영화,<!--[[--image0--]]//--> \n[매일경제 스타투데이 양소영 기자]...,NaN
239821,4407591,2020-03-01,51108.0,영화,<!--[[--image0--]]//--> \n[매일경제 스타투데이 양소영 기자]...,NaN
...,...,...,...,...,...,...
260296,4441458,2020-03-31,230643.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 허은경 객원기자]...,NaN
260297,4441459,2020-03-31,231318.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 허은경 객원기자]...,NaN
260298,4441460,2020-03-31,231505.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN
260299,4441462,2020-03-31,233040.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN


In [17]:
news_202003['body1'] = news_202003['body'].fillna('없음').apply(lambda x : re.sub('[0-9]+', '', x))
news_202003['body1'] = news_202003['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
news_202003['body1'] = news_202003['body1'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》◆■\n]', '', x))

<ipython-input-17-23314b223089>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202003['body1'] = news_202003['body'].fillna('없음').apply(lambda x : re.sub('[0-9]+', '', x))
<ipython-input-17-23314b223089>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202003['body1'] = news_202003['body1'].apply(lambda x : re.sub('[A-Za-z]+', '', x))
<ipython-input-17-23314b223089>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [19]:
news_202003

,no,pubdate,pubtime,title1,body,keywords,body1
0,2304119,2020-03-12,0.0,사회일반,NaN,NaN,없음
239818,4407584,2020-03-01,2159.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 원태성 객원기자]...,NaN,매일경제 스타투데이 원태성 객원기자장성규가 전현무의 독설을 폭로했다일 밤 방송된...
239819,4407589,2020-03-01,41109.0,영화,<!--[[--image0--]]//--> \n[매일경제 스타투데이 양소영 기자...,NaN,매일경제 스타투데이 양소영 기자 홍상수 감독이 제회 베를린국제영화제에서 감독상...
239820,4407590,2020-03-01,43011.0,영화,<!--[[--image0--]]//--> \n[매일경제 스타투데이 양소영 기자]...,NaN,매일경제 스타투데이 양소영 기자 홍상수 감독이 제회 베를린국제영화제에서 낭보를 ...
239821,4407591,2020-03-01,51108.0,영화,<!--[[--image0--]]//--> \n[매일경제 스타투데이 양소영 기자]...,NaN,매일경제 스타투데이 양소영 기자 홍상수 감독과 김민희가 함께한 신작 도망친 여자...
...,...,...,...,...,...,...,...
260296,4441458,2020-03-31,230643.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 허은경 객원기자]...,NaN,매일경제 스타투데이 허은경 객원기자 아내의 맛 홍현희제이쓴 부부가 낙지 체취에 ...
260297,4441459,2020-03-31,231318.0,방송/TV,<!--[[--image0--]]//--> [매일경제 스타투데이 허은경 객원기자]...,NaN,매일경제 스타투데이 허은경 객원기자 사람이 좋다 가수 박혜경이 어려움을 딛고 인...
260298,4441460,2020-03-31,231505.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN,매일경제 스타투데이 서지경 객원기자 불타는 청춘의 박선영이 재치 있는 농담으...
260299,4441462,2020-03-31,233040.0,방송/TV,<!--[[--image0--]]//--> \n[ 매일경제 스타투데이 서지경 객원...,NaN,매일경제 스타투데이 서지경 객원기자 불타는 청춘의 구본승과 안혜경이 낚시에 ...


In [20]:
news_202003['freq'] = news_202003['body1'].apply(lambda x : okt.nouns(x))

<ipython-input-20-5f4da61b4d5f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202003['freq'] = news_202003['body1'].apply(lambda x : okt.nouns(x))


In [21]:
news_202003['freq_10'] = news_202003['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))

<ipython-input-21-ccf746a531a7>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  news_202003['freq_10'] = news_202003['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))
<ipython-input-21-ccf746a531a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202003['freq_10'] = news_202003['freq'].apply(lambda x : list(pd.Series(x).value_counts().head(10).index))


In [22]:
news_202003['freq_10'] = news_202003['freq_10'].apply(lambda x : ",".join(x))

<ipython-input-22-75ac29544a53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_202003['freq_10'] = news_202003['freq_10'].apply(lambda x : ",".join(x))


In [23]:
news_202003.to_excel("news_202003.xlsx")